<a href="https://colab.research.google.com/github/raheelsiddiqi2013/fruit-grading/blob/main/LocalSindhiGradingSecondNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grading of Local Sindhi Guava
This notebook is written and executed by **Raheel Siddiqi, PhD**. Fruit grading of **Local Sindhi** guava is performed. It is a multi-class classification problem with three classes: "Green", "Mature Green", and "Ripe". The dataset consists of 711 images with a train-validation-test split of 70-15-15. The pre-trained VGG16 model is used for feature extraction, while a densely connected network is used for final classification. A test set classification accuracy of **98.96%** has been achieved.

## Connecting to Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/MyResearch/FruitGrading/LocalSindhi"

Test  Train  Validation


## Importing Relevant Libraries

In [ ]:
# Import
import numpy as np
from PIL import Image
import os

from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import VGG16

## Loading Data

In [ ]:
# Create Generator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2
                                   )

validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)


# Create Data Loaders
train_loader = train_datagen.flow_from_directory(
    directory = "/content/drive/MyDrive/MyResearch/FruitGrading/LocalSindhi/Train",
    target_size = (224, 224),
    batch_size = 32,
    shuffle = True
)

validation_loader= validation_datagen.flow_from_directory(
    directory = "/content/drive/MyDrive/MyResearch/FruitGrading/LocalSindhi/Validation",
    target_size = (224, 224),
    batch_size = 32,
    shuffle = True
)

test_loader = test_datagen.flow_from_directory(
    directory = "/content/drive/MyDrive/MyResearch/FruitGrading/LocalSindhi/Test",
    target_size = (224, 224),
    batch_size = 32,
    shuffle = True
)

Found 498 images belonging to 3 classes.
Found 107 images belonging to 3 classes.
Found 106 images belonging to 3 classes.


## Dataset Classes

In [ ]:
train_loader.class_indices

{'Green': 0, 'Mature Green': 1, 'Ripe': 2}

## Hyperparameter Settings and Model Compilation

In [ ]:
from keras.callbacks import ModelCheckpoint
import keras

optimizer = keras.optimizers.Adam(lr=0.0001)
model = Sequential()
model.add(VGG16(include_top = False, pooling = "avg"))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['categorical_accuracy'])

filepath="LocalSindhi_VGG16_fruit_image_classification_3_classes.h5"
checkpoint = ModelCheckpoint(filepath, monitor="val_loss", save_best_only=True)
callbacks_list = [checkpoint]

58889256/58889256 [==============================] - 4s 0us/step


## Model Training

In [ ]:
import math

model_train_history = model.fit_generator(
    generator = train_loader,
    steps_per_epoch = math.ceil(498//32),
    epochs = 100,
    callbacks=callbacks_list,
    validation_data = validation_loader,
    validation_steps = math.ceil(107//32)
)

<ipython-input-10-6e726d79eb7f>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_train_history = model.fit_generator(


Epoch 1/100
15/15 [==============================] - ETA: 0s - loss: 1.3955 - categorical_accuracy: 0.3863 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


15/15 [==============================] - 493s 31s/step - loss: 1.3955 - categorical_accuracy: 0.3863 - val_loss: 1.0653 - val_categorical_accuracy: 0.3750
Epoch 2/100
15/15 [==============================] - 16s 1s/step - loss: 1.0212 - categorical_accuracy: 0.4764 - val_loss: 1.0557 - val_categorical_accuracy: 0.5000
Epoch 3/100
15/15 [==============================] - 15s 954ms/step - loss: 1.0791 - categorical_accuracy: 0.4485 - val_loss: 0.9717 - val_categorical_accuracy: 0.5208
Epoch 4/100
15/15 [==============================] - 15s 950ms/step - loss: 0.9820 - categorical_accuracy: 0.5150 - val_loss: 0.9446 - val_categorical_accuracy: 0.5104
Epoch 5/100
15/15 [==============================] - 16s 1s/step - loss: 0.9344 - categorical_accuracy: 0.4721 - val_loss: 0.9104 - val_categorical_accuracy: 0.4896
Epoch 6/100
15/15 [==============================] - 15s 943ms/step - loss: 0.8873 - categorical_accuracy: 0.5365 - val_loss: 0.8478 - val_categorical_accuracy: 0.5312
Epoch 7/100

## Model Testing

In [ ]:
from keras.models import load_model
best_model = load_model('LocalSindhi_VGG16_fruit_image_classification_3_classes.h5')
steps_test=int(106/32)
result = best_model.evaluate_generator(test_loader, steps=steps_test,verbose=1)
print("Test-set accuracy: {0:.2%}".format(result[1]))

<ipython-input-13-de7246e46616>:4: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  result = best_model.evaluate_generator(test_loader, steps=steps_test,verbose=1)


3/3 [==============================] - 1s 309ms/step - loss: 0.0867 - categorical_accuracy: 0.9896
Test-set accuracy: 98.96%
